In [1]:
#import all required libraries
from configparser import ConfigParser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from datetime import date
import pandas as pd



#Add User agent to scrape the site

service = ChromeService(executable_path=ChromeDriverManager().install())
opts = ChromeOptions()


opts.add_argument(("user-agent=Mozilla/5.0"
                   +" (Macintosh; Intel Mac OS X 10_15_7)"
                   +" AppleWebKit/537.36 (KHTML, like Gecko)"
                   +" Chrome/88.0.4324.96 Safari/537.36"))


In [2]:
TODAY = (f"{date.today().month}/"
         +f"{date.today().day}/"
         +f"{date.today().year}")

In [3]:
driver = webdriver.Chrome(options=opts, service=service)

driver.get('https://jailbreakbrewing.com')
button = driver.find_element(By.TAG_NAME, 'button')
button.click()
time.sleep(10)
jail_cols = driver.find_elements(By.CLASS_NAME,'whatsontaphdr')
jailbreak = pd.DataFrame(columns = [
    'Brewery',
    'Date_Retrieved']+[col.text for col in jail_cols])
jail_beers = driver.find_elements(By.CLASS_NAME,'whatsontap')
i = 0
for col in jailbreak.columns[2:]:
    jailbreak[col] = [j.text for j in jail_beers[i::4]]
    i +=1
jailbreak['Brewery'] = 'Jailbreak Brewing'
jailbreak['Date_Retrieved'] = TODAY
jailbreak['ABV'] = jailbreak['ABV'].str.replace('%','').astype(float)

In [5]:
# driver.quit()

In [16]:
jailbreak

,Brewery,Date_Retrieved,Name,Style,ABV,IBU
0,Jailbreak Brewing,1/29/2023,Feed The Monkey,Orange Hefeweizen,5.6,14
1,Jailbreak Brewing,1/29/2023,The Infinite,Amber Ale,5.0,19
2,Jailbreak Brewing,1/29/2023,Clearly Righteous,West Coast-Style IPA,6.7,N/A
3,Jailbreak Brewing,1/29/2023,Oatmeal Stout: It's What's For Breakfast,Oatmeal Stout,4.5,N/A
4,Jailbreak Brewing,1/29/2023,Hazy Ryeder,Hazy Rye IPA,6.8,N/A
5,Jailbreak Brewing,1/29/2023,The White Russian,Coffee Cream Ale,5.5,13
6,Jailbreak Brewing,1/29/2023,Vienna Lager,Vienna Lager,5.5,N/A
7,Jailbreak Brewing,1/29/2023,Big Punisher,Double IPA,8.5,93
8,Jailbreak Brewing,1/29/2023,Doppelbock,German Lager,7.5,N/A
9,Jailbreak Brewing,1/29/2023,Czech Dark Lager,Czech Dark Lager,4.8,N/A


In [17]:
driver.get("https://www.blackflagbrewingco.com")
button = driver.find_element(By.ID,"ageYes")
button.click()

In [18]:
blackflag = pd.DataFrame(columns = ['Name','Style','ABV','IBU'])

In [23]:
blackflag = pd.DataFrame(columns = ['Brewery','Date_Retrieved','Name','Style',
                                   'Description','ABV'])
elements = driver.find_element(By.CLASS_NAME,'col-md-12')
flag_beers = elements.find_elements(By.TAG_NAME,'td')

i = 0
for col in blackflag.columns[2:]:
    blackflag[col] = [j.text for j in flag_beers[i::5]]
    i +=1
blackflag['Brewery'] = 'Black Flag Brewing'
blackflag['Date_Retrieved'] = TODAY
blackflag['ABV'] = blackflag.ABV.astype(float)


In [24]:
blackflag

,Brewery,Date_Retrieved,Name,Style,Description,ABV
0,Black Flag Brewing,1/29/2023,Gelato Terpene Kashmir,IPA - New England / Hazy,We took one sixtel of our Super Kashmir hazy I...,7.0
1,Black Flag Brewing,1/29/2023,Super Kashmir,IPA - New England / Hazy,SUPER KASHMIR is a revamped version of a Black...,7.0
2,Black Flag Brewing,1/29/2023,Barrel-Aged Cuvée,Stout - Other,We took a rich 11% ABV base stout and aged it ...,11.5
3,Black Flag Brewing,1/29/2023,Red Velvet Mage,Stout - Imperial / Double Pastry,Red Velvet Mage is a 12% ABV imperial stout fo...,12.0
4,Black Flag Brewing,1/29/2023,Dubbel Trubbel,Belgian Dubbel,Brewer Alex whipped up this Belgian Dubbel fea...,5.5
5,Black Flag Brewing,1/29/2023,Cobbler: Peach/Apricot/Black Currant,Sour - Fruited,Our Cobbler series of fruited sours is back wi...,6.0
6,Black Flag Brewing,1/29/2023,Particle Horizon,Barleywine - American,Particle Horizon is our newest barleywine brew...,10.3
7,Black Flag Brewing,1/29/2023,Crispy Boy: Vienna Lager,Lager - Vienna,Please say hello to one of Austria's finest ex...,5.5
8,Black Flag Brewing,1/29/2023,Oatmeal Dreme Pie,Stout - Other,The next in our line-up of savory yet sweet 6%...,6.0
9,Black Flag Brewing,1/29/2023,Sun Run,IPA - New England / Hazy,Citra and Mosaic hops combine to give you a sh...,6.5


In [12]:
driver.quit()

[WDM] - Downloading: 100%|██████████| 8.72M/8.72M [05:21<00:00, 28.5kB/s]


In [7]:
driver.get('https://loneoakbrewingco.com/brews/')

In [8]:
loneoak = driver.find_elements(By.CLASS_NAME,'food-items')
for e in loneoak:
    print(e.text)

In [33]:
pd.concat([blackflag,jailbreak],axis = 0,
          ignore_index = True)[
    ['Brewery','Date_Retrieved',
     'Name','Style','ABV',
     'Description','IBU']
    ].sort_values(by = 'ABV')

,Brewery,Date_Retrieved,Name,Style,ABV,Description,IBU
26,Jailbreak Brewing,1/29/2023,Spruce Lee Funk,Spruce Tip + Cranberry Sour,4.0,NaN,N/A
19,Jailbreak Brewing,1/29/2023,Oatmeal Stout: It's What's For Breakfast,Oatmeal Stout,4.5,NaN,N/A
12,Black Flag Brewing,1/29/2023,Seltzer Bar,Hard Seltzer,4.5,11/11/22 - We have revamped our Seltzer Bar to...,NaN
10,Black Flag Brewing,1/29/2023,K Kapowski,Lager - Amber,4.8,,NaN
25,Jailbreak Brewing,1/29/2023,Czech Dark Lager,Czech Dark Lager,4.8,NaN,N/A
30,Jailbreak Brewing,1/29/2023,Indian Lager ( $5 Can Pour),Rice Lager,5.0,NaN,N/A
17,Jailbreak Brewing,1/29/2023,The Infinite,Amber Ale,5.0,NaN,19
13,Black Flag Brewing,1/29/2023,Lager,Lager - American,5.0,"This super clean lager features Pilsner, Chit,...",NaN
14,Black Flag Brewing,1/29/2023,Vanilla Porter,Porter - Other,5.1,5.1% Vanilla Porter. Smooth with a touch of ro...,NaN
22,Jailbreak Brewing,1/29/2023,Vienna Lager,Vienna Lager,5.5,NaN,N/A


In [27]:
driver.quit()

In [39]:
driver.get("https://www.saintsrowbeer.com/saintsrow-taplist")

In [74]:
elements = driver.find_elements(By.CLASS_NAME,'item-name')
print(len(elements))
[e.find_element(By.TAG_NAME,'a').text for e in elements]

16


['Morning Offering',
 'Hopostolic Succession',
 'Saint Ronin',
 'Pumpkin Saints Latte',
 'I Believe',
 'Yuletide',
 'Que Chilero',
 'Neo Space Ghost',
 '5 Year Harvest',
 'Olde Towne: Schwarzbier',
 'Vanilla Connection',
 'Free State Pilsner',
 'That Irish Goodbye',
 'Rainbow Connection',
 'Dark Necessities 2022',
 'Ongoing Saga (Nitro)']

In [73]:
len(elements)

32

In [72]:
elements = driver.find_elements(By.CLASS_NAME,'item-style')
[e.text for e in elements[::2]]

['Mocha Blonde',
 'DDH Saison',
 'Sake Pale w/Lemon & Ginger',
 'Pumpkin Spice Horchata',
 'Lager - American Light',
 'Sahti Winter Warmer',
 'Central American Amber Lager',
 'White Ale',
 'Red Ale - Imperial',
 'Schwarzbier',
 'Orange Creamsicle NEIPA',
 'German Pilsner',
 'Stout - Irish Dry',
 'House NEIPA',
 'Imperial Coffee Milk Stout',
 'Stout - Imperial']

In [77]:
elements = driver.find_elements(By.CLASS_NAME,'item-abv')
[e.text for e in elements]

['6% ABV',
 '7% ABV',
 '5.5% ABV',
 '5% ABV',
 '4.5% ABV',
 '8% ABV',
 '5% ABV',
 '5% ABV',
 '8% ABV',
 '4.8% ABV',
 '6.5% ABV',
 '5% ABV',
 '4.7% ABV',
 '6.5% ABV',
 '9% ABV',
 '8.5% ABV']

In [81]:
elements = driver.find_elements(By.CLASS_NAME,"link-font-color show-less item-title-color show-more")
[e.text for e in elements]

[]

In [65]:
for e in elements:
    x = e.find_element(By.TAG_NAME,'a')
    print(x.text)

Morning Offering
Hopostolic Succession
Saint Ronin
Pumpkin Saints Latte
I Believe
Yuletide
Que Chilero
Neo Space Ghost
5 Year Harvest
Olde Towne: Schwarzbier
Vanilla Connection
Free State Pilsner
That Irish Goodbye
Rainbow Connection
Dark Necessities 2022
Ongoing Saga (Nitro)


In [57]:
elements = driver.find_elements(By.CLASS_NAME,'item-name')
saint_beers = elements.find_elements(By.TAG_NAME,'a')

In [63]:
[s.text for s in saint_beers]

['Morning Offering']

In [59]:
saint_beers

[<selenium.webdriver.remote.webelement.WebElement (session="7f2b22117d0450055bbc7169fa9fa7b0", element="288196a0-6d7d-44a3-b507-303b21be3907")>]

In [4]:
driver.quit()